In [3]:
import cv2
import numpy as np
import pickle
from matplotlib import pyplot as plt

In [7]:
def skin_detection(img):
    
    skinLow = np.array([0,48,80],np.uint8)
    skinHigh = np.array([20,255,255],np.uint8)
    img = cv2.GaussianBlur(img,(5,5),0)
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(imgHSV,skinLow,skinHigh)
    result = cv2.bitwise_and(img,img,mask=mask)
    imgGray = cv2.cvtColor(result,cv2.COLOR_BGR2GRAY)
    
    return imgGray

In [8]:
winSize = (128,128)
blockSize = (16,16)
blockStride = (4,4)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,
                        derivAperture,winSigma,histogramNormType,
                        L2HysThreshold,gammaCorrection,nlevels)

In [13]:
img = pickle.load(open("img_save.p","rb"))

In [15]:
pre = []
hog_data = []
hog_data_flat = []

for n in img:
    pre.append(skin_detection(n))
for n in pre:
    hog_data.append(hog.compute(n))
for n in hog_data:
    hog_data_flat.append(n.flatten())

In [16]:
hog_data_flat[0]

array([0.        , 0.        , 0.        , ..., 0.4300235 , 0.35288692,
       0.10567046], dtype=float32)

In [23]:
x = np.float32(hog_data_flat).reshape(-1,30276)
y = np.float32(np.repeat(np.arange(24),19)[:,np.newaxis])

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

clf = svm.SVC()
cross_val = cross_val_score(clf, x, y.ravel(), cv=10)
cross_val

array([0.93478261, 0.76086957, 0.58695652, 0.65217391, 0.76086957,
       0.84782609, 0.84444444, 0.91111111, 0.84444444, 0.8       ])

In [32]:
print(np.mean(cross_val))

0.7943478260869565
